<a href="https://colab.research.google.com/github/hobg0blin/stylegan3/blob/main/Mode_Collapse_stylegan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN3 from NVIDIA

This is adapted from [this Colab](https://colab.research.google.com/drive/1Nal3M-wjv6BeIgyTgvxhaccPFGEP6cbk?usp=sharing), but using the most up-to-date version of SG3 and code adapted for my own purposes. I've found specifying Torch version to be a requisite, and tweaked some of the original NVIDIA code to work more within python rather than requiring command line calls.

# Setup

In [1]:
!pip install einops ninja gdown click

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 22.6 MB/s eta 0:00:00


In [2]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 80.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.6 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.1 MB/s eta 0:00:

In [3]:
# Connect Google Drive
from google.colab import drive
drive.flush_and_unmount()
!rm -rf /content/drive
drive.mount('/content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [4]:
import os
if os.path.isdir('/content/drive/MyDrive/WIP/stylegan3/'):
    %cd '/content/drive/MyDrive/WIP/stylegan3/'
else:
    !git clone https://github.com/NVlabs/stylegan3.git /content/drive/MyDrive/WIP/stylegan3/
    %cd '/content/drive/MyDrive/WIP/stylegan3/'


/content/drive/MyDrive/WIP/stylegan3


If you wanted to download the current dataset execute below code

In [5]:
#!bash /content/drive/MyDrive/WIP/stargan-v2-download/download.sh afhq-dataset


# Ouroboros
Loop training and dataset generation, feed back into model.

In [ ]:
import os
#import sys
#sys.path.append('/content/drive/MyDrive/WIP/stylegan3/')
from train import train_model
from gen_images import generate_images
from dataset_tool import convert_dataset

latest_output = "latest_output"
batch_size = 16
batch_gpu_size = 8 # Batch size for GPU

dataset_path = '/content/drive/MyDrive/WIP/stylegan3/datasets/'
#resume_from = 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-1024x1024.pkl'  #Can be the link from nvidia
latest_model ='/content/drive/MyDrive/WIP/stylegan3/models/30k_snapshot.pkl'


gamma_value = 50.0 #Adjustable R1 regularization weight
snapshot_count = 8
for i in range(0, 5):
  print('---generating_images---')
  generate_images(outdir="/content/drive/MyDrive/WIP/stylegan3/latest_output", truncation_psi=1, seeds=range(5000, 10000), network_pkl=latest_model, noise_mode='const', translate=[0,0], rotate=0, class_idx=None)
  generate_images(outdir="/content/drive/MyDrive/WIP/stylegan3/sample_outputs/run_" + i, seeds=range(0, 10),  truncation_psi=1, network_pkl=latest_model, noise_mode='const', translate=[0,0], rotate=0, class_idx=None)
  print('---prepping dataset---')
  convert_dataset(source="/content/drive/MyDrive/WIP/stylegan3/" + latest_output,dest=dataset_path + latest_output + ".zip", resolution='1024x1024')
  os.system("rm -rf /content/drive/MyDrive/WIP/stylegan3/latest_output/*")
  print('---training model---')
  train_model(outdir=f"/content/drive/MyDrive/WIP/stylegan3/training_${i}", cfg="stylegan3-r", data=f"${dataset_path + latest_output}.zip", gpus=1, batch=batch_size, batch_gpu=batch_gpu_size, gamma=gamma_value, kimg=1, snap=snapshot_count, resume=latest_model)
  latest_model = f"/content/drive/MyDrive/WIP/stylegan3/training_${i}/network-snapshot.pkl"


---generating_images---
Loading networks from "/content/drive/MyDrive/WIP/stylegan3/models/30k_snapshot.pkl"...
Generating image for seed 5000 (0/5000) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
Generating image for seed 5001 (1/5000) ...
Generating image for seed 5002 (2/5000) ...
Generating image for seed 5003 (3/5000) ...
Generating image for seed 5004 (4/5000) ...
Generating image for seed 5005 (5/5000) ...
Generating image for seed 5006 (6/5000) ...
Generating image for seed 5007 (7/5000) ...
Generating image for seed 5008 (8/5000) ...
Generating image for seed 5009 (9/5000) ...
Generating image for seed 5010 (10/5000) ...
Generating image for seed 5011 (11/5000) ...
Generating image for seed 5012 (12/5000) ...
Generating image for seed 5013 (13/5000) ...
Generating image for seed 5014 (14/5000) ...
Generating image for seed 5015 (15/5000) ...
Generating image for seed 5016 (16/5000) ...
Generating image for 

In [ ]:
""